# 1. Connect to Azure Machine Learning Workspace

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

## 1.2 Configure workspace details and get a handle to the workspace

In [ ]:
# enter details of your Azure Machine Learning workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AZUREML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Create Online Endpoint

In [ ]:
# Define an endpoint name
endpoint_name = "basic-chat-endpoint-sdk"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="this is a sample endpoint",
    auth_mode="key"
)

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 3. Create Deployment

In [ ]:
model = Model(path="../../../../examples/flows/chat/basic-chat",
              properties={"azureml.promptflow.source_flow_id": "basic-chat", # endpoint detail UI Test tab needs this property to know it's from prompt flow
                        # Following are properties only for chat flow 
                          "azureml.promptflow.mode": "chat",
                          "azureml.promptflow.chat_input": "question",
                          "azureml.promptflow.chat_output": "answer"
                          })
env = Environment(
    image="mcr.microsoft.com/azureml/promptflow/promptflow-runtime:20230801.v1",
    inference_config={
            "liveness_route": {"path": "/health", "port": 8080},
            "readiness_route": {"path": "/health", "port": 8080},
            "scoring_route": {"path": "/score", "port": 8080},
        },
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    instance_type="Standard_E16s_v3",
    instance_count=1,
    environment_variables={"PROMPTFLOW_RUN_MODE": "serving",
                            # currently it is for pulling connections from workspace
                           "PRT_CONFIG_OVERRIDE": "deployment.subscription_id=<subscription_id>,deployment.resource_group=<resource_group>,deployment.workspace_name=<workspace_name>,deployment.endpoint_name=<endpoint_name>,deployment.deployment_name=<deployment_name>"
                           },
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# 4. Test the endpoint with sample data

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="../cli/sample-request.json",
)